In [ ]:
# 성공

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains  # 마우스 액션 사용
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException
import time
import json
import time
import traceback
from selenium.common.exceptions import NoSuchWindowException, NoSuchElementException, TimeoutException


# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'
service = Service(executable_path=driver_path)

# Chrome 실행
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    print("쿠키 로드 성공")
except Exception as e:
    print(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Perf/51026?Gcode=009_300')

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[10]/div/div[4]/a[4]'))
    )
    book_button.click()  # 예매하기 버튼 클릭
    print("예매하기 버튼 클릭 완료")
except Exception as e:
    print(f"예매하기 버튼 클릭 오류: {e}")

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        print(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            print("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    print(f"새 창 제어 오류: {e}")

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-12-29"))  # 날짜 선택
    )
    date_button.click()
    print("날짜 선택 완료")
except Exception as e:
    print(f"날짜 선택 오류: {e}")

# 회차 선택 (1회차, 13시 00분 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용) 
    )
    time_button.click()
    print("회차 선택 완료")
except Exception as e:
    print(f"회차 선택 오류: {e}")

# 좌석 선택
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnSeatSelect"]'))  # 좌석 선택 버튼 (XPath 사용) 
    )
    seat_button.click()
    print("좌석 선택 완료")
except Exception as e:
    print(f"좌석 선택 오류: {e}")


def monitor_and_select_seat():
    attempts = 0
    while attempts < max_attempts:
        try:
            print(f"Attempt {attempts + 1} out of {max_attempts}")

            # 기본 컨텐츠로 전환
            driver.switch_to.default_content()
            print("Switched to default content")

            # 좌석 선택 프레임으로 전환
            iframe_seat = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
            )
            driver.switch_to.frame(iframe_seat)
            print("Switched to iframe 'ifrmSeatFrame'")

            # 선택 가능한 좌석 찾기
            seats = driver.find_elements(By.XPATH, "//div[starts-with(@id, 't') and @name='tk' and @grade]")
            if seats:
                print(f"{len(seats)}개의 선택 가능한 좌석을 찾았습니다.")
                seats[0].click()  # 첫 번째 좌석 선택
                print("좌석을 선택했습니다.")

                # 좌석 클릭 후 잠시 대기
                time.sleep(1)

                # 알림 창 처리 (필요한 경우)
                try:
                    WebDriverWait(driver, 3).until(EC.alert_is_present())
                    alert = driver.switch_to.alert
                    print(f"Alert text: {alert.text}")
                    alert.accept()
                    print("Alert accepted.")
                except TimeoutException:
                    print("No alert appeared after seat selection.")

                # 좌석 선택 후 기본 컨텐츠로 전환
                driver.switch_to.default_content()
                print("Switched to default content after clicking the seat")

                # 필요한 경우, 다시 프레임 전환
                # 여기서는 'ifrmSeatFrame' 프레임으로 다시 전환합니다.
                iframe_seat = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
                )
                driver.switch_to.frame(iframe_seat)
                print("Switched to iframe 'ifrmSeatFrame' to find the confirm button")

                # "좌석선택완료" 버튼 찾기
                try:
                    # XPath를 수정하여 버튼 찾기
                    confirm_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
                    )
                    confirm_button.click()
                    print("좌석선택완료 버튼을 클릭했습니다.")
                except Exception as e:
                    print(f"버튼 클릭에 실패했습니다: {e}")
                    traceback.print_exc()
                    return False

                # 좌석 선택 완료 후 함수 종료
                return True  # 성공적으로 좌석 선택 및 확인 버튼 클릭
            else:
                print(f"선택 가능한 좌석을 찾지 못했습니다. {monitoring_interval}초 후에 다시 시도합니다.")
        except NoSuchWindowException as e:
            print(f"No such window exception occurred: {e}")
            print("Available window handles:", driver.window_handles)
            traceback.print_exc()
            return False  # 창이 닫혔으므로 함수 종료
        except Exception as e:
            print(f"좌석 선택 시 문제가 발생했습니다: {e}")
            traceback.print_exc()
            print("Available window handles:", driver.window_handles)

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)
        attempts += 1

    return False  # 최대 시도 횟수 초과

# 좌석 선택 모니터링 시작
if monitor_and_select_seat():
    print("좌석 선택에 성공했습니다.")
else:
    print("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")
